# LLM 모델 파인튜닝하기

In [ ]:
# 환경변수 불러오기
from dotenv import load_dotenv

load_dotenv()

# 허깅페이스 로그인
from huggingface_hub import login
import os

login(token=os.getenv("HUGGINGFACE_API_KEY"))

In [ ]:
# Pytorch 공식 홈페이지에서 CCUDA 버전에 맞춰서 실시
import torch

print(torch.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda


# 1. 모델 불러오기

In [ ]:
# 양자화 할까말까 ? -> 양자화를 하는 이유: 메모리 사용량을 조금이라도 줄이려고 큰 숫자들을 자르거나 작은 숫자로 변경해줌.
# parameter: quantization_config=BitsAndBytesConfig()
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2b-it"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    dtype=torch.bfloat16,
    attn_implementation="eager",  # 어텐션 계산 방법의 일종(Gemma - eager 기억하기 -> 무조건 기억하기)
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

c:\LLMproject\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google--gemma-2b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# 2. 토크나이저 불러오기

In [16]:
# 텍스트가 있을 때 텐서로 잘 바꿔주는 지 확인하기
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
text = "안녕하세요"
tokenized_text = tokenizer(text)
print(tokenized_text)
decoded_text = tokenizer.decode(
    tokenized_text["input_ids"], skip_special_tokens=True
)  # skip_special_tokens 시작이랑 끝 제거 (True로 했을때, <bos> 가 안뜸)
print(decoded_text)

{'input_ids': [2, 238179, 243415, 204551], 'attention_mask': [1, 1, 1, 1]}
안녕하세요


In [ ]:
# 나, 는, 학, 교, 에, 갑, 니, 다  8개
# 나, 는, 학, 교, 에, 갑, 니, 다, pad, pad 10개
# 1,  1,  1, 1,  1,  1,  1,  1,   0,   0

In [ ]:
messages = [
    [
        {"role": "user", "content": "친절하게 답변해주세요\n 안녕하세요"},
        {"role": "assistant", "content": "안녕하세요. 반가워요"},
    ],
]

tokenized_messages = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    tokenize=True,
    return_dict=True,
    return_tenseors="pt",
)
print(tokenized_messages)
decoded_messages = tokenizer.decode(
    tokenized_messages["input_ids"][0], skip_special_token=False
)

{'input_ids': [[2, 106, 1645, 108, 240471, 240046, 101969, 235248, 241305, 239042, 237138, 237014, 96673, 108, 70685, 243415, 204551, 107, 108, 106, 2516, 108, 238179, 243415, 204551, 235265, 78821, 236361, 239779, 237526, 107, 108]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# 3. 학습 준비

### 1) Huggingface 에서 제공하는 데이터셋 불러오기

In [ ]:
from datasets import load_dataset

dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [ ]:
dataset["train"].select(range(10))  # 10개만 가져오기
dataset["train"].shuffle(seed=42).select(range(10))  # 랜덤으로 섞은 뒤 10개만 가져오기

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 10
})

In [ ]:
sample_dataset = dataset["train"].select(
    range(10)
)  # 10개의 데이터 ## 조회해보기: sample_dataset[0]
sample_dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 10
})

In [22]:
sample_dataset[0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

### 2) 커스텀 데이터 만들기

In [ ]:
# instruct 모델을 파인튜닝하는 이유??? --- 다음 신문을 요약해주세요. 그리고 핵심 키워드 5개를 뽑아주세요.
# STEP1. 위에서 불러온 데이터를 가지고 keywords를 뽑아주는 데이터를 만들어야 한다.
# STEP2. document, summary, keywords를 가지고 messages 형태의 데이터를 만든다.
# messages = [
#     [
#         {
#             "role": "user",
#             "content":
#         },
#         {
#             "role": "assistant",
#             "content":
#         },
#     ],
# ]

In [ ]:
# STEP1

from openai import OpenAI

client = OpenAI()


# {'date': '2022-07-03 17:14:37',
#  'category': 'economy',
#  'press': 'YTN ',
#  'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
#  'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
#  'link': 'https://n.news.naver.com/mnews/article/052/0001759333?sid=101',
#  'summary': '올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했다.'}
def get_keywords(data):
    messages = [
        {
            "role": "user",  # user 에 다 넣어서 표현한거임(어차피 모델 활용만 하는 것이기 떄문에)
            "content": f"뉴스에서 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] {data['document']}\n\n[예시] 키워드1, 키워드2, 키워드3, 키워드4, 키워드5",
        }
    ]
    # output = pipe(messages, max_new_tokens=128)
    output = client.chat.completions.create(model="gpt-5-nano", messages=messages)
    output = output.choices[0].message.content

    data["keywords"] = output

    return data

In [24]:
sample_dataset[0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [26]:
get_keywords(sample_dataset[0])

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [27]:
sample_dataset = sample_dataset.map(get_keywords)
sample_dataset

Parameter 'function'=<function get_keywords at 0x000002BD9A182C20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'keywords'],
    num_rows: 10
})

### 3) 학습 데이터로 만들기

In [31]:
# STEP2
def make_prompt(data):
    messages = [
        {
            "role": "user",
            "content": f"뉴스를 요약해주세요.\n\n 그리고 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] {data['document']}\n\n[예시]키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5",
        },
        {
            "role": "assistant",
            "content": f"[키워드]: {data['keywords']}\n\n[뉴스요약]: {data['summary']}",
        },
    ]

    return {
        "text": tokenizer.apply_chat_template(
            messages, add_generation_prompt=False, tokenize=False
        )
        + "<eos>"
    }

In [32]:
sample_dataset[0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [33]:
make_prompt(sample_dataset[0])

{'text': '<bos><start_of_turn>user\n뉴스를 요약해주세요.\n\n 그리고 핵심 키워드를 예시와 같이 명사로 5개 추출해주세요\n\n[뉴스전문] 앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.\n\n[예시]키워드: 키워드1, 키워드2, 키워드3, 키워드4, 키워드5<end_of_turn>\n<start_of_turn>mod

In [34]:
sample_dataset = sample_dataset.map(make_prompt)
sample_dataset

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'keywords', 'text'],
    num_rows: 10
})

# 4. 학습

## 1) 학습 설정

## LoraConfig 주요 파라미터 정리

| 파라미터 | 의미 | 권장 기본값 | 커스텀 가이드 |
|----------|------|-------------|----------------|
| r | LoRA 어댑터의 차원(랭크) | 16 | VRAM 적으면 8, 성능↑ 원하면 32 |
| lora_alpha | LoRA 스케일링 계수 (실질 영향력은 alpha/r) | 32 (보통 r*2) | 성능 부족 → 크게, 안정성 필요 → 작게 |
| lora_dropout | 어댑터 입력에 드롭아웃 적용 (학습 시만) | 0.05 | 데이터 적고 과적합 우려 → 0.1 ↑, 데이터 충분 → 0 |
| bias | bias 파라미터 포함 여부 | "none" | 거의 항상 "none" 사용 |
| task_type | 적용할 태스크 유형 | "CAUSAL_LM" | 생성형 LM이면 그대로 사용 |
| target_modules | LoRA를 적용할 레이어 이름 패턴 | ["q_proj","k_proj","v_proj","o_proj"] | 시작은 어텐션만 → 필요시 FFN까지(gate/up/down_proj) 확대 |
| modules_to_save | LoRA 외에 저장할 모듈 | None | 추가 성능 필요 시 ["lm_head"] 같이 지정 |

In [36]:
# 전체 가중치를 업데이트 하는 방식 : Full FT
# 가중치 일부만 바꾸는 방식: PEFT(parameter-Efficient FT) 학습을 위한 설정
# -- loRA(저차원행렬의 곱셈) "기존의 가중치는 고정하고 Adapter 의 가중치를 업데이트 한다"
# uv add peft
from peft import LoraConfig

lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

## 2) 학습

In [38]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir = "./outputs",
    num_train_epochs=2,
    max_steps=2,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    logging_steps=1
)

## SFTTrainer 주요 파라미터
| 파라미터 | 설명 | 부연 설명 |
|----------|------|----------|
| model | 학습할 모델 | 동일 |
| tokenizer | 토크나이저 | 동일 |
| train_dataset | 학습 데이터셋 | 동일 |
| eval_dataset | 검증 데이터셋 | 동일 |
| args | SFTConfig 객체 | TrainingArguments 대신 SFTConfig 사용 |
| formatting_func | 데이터 포맷팅 함수 | 원시 데이터 → 대화형 텍스트 변환 (ex. instruction/output) |
| dataset_text_field | 텍스트 열 이름 | SFTConfig와 연결됨 |
| max_seq_length | 최대 입력 길이 | 동일 |
| packing | 여러 샘플 묶기 | 동일 |
| peft_config | LoRA/QLoRA 설정 | 동일 |

In [ ]:
# uv add trl

from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=sample_dataset,
    args=args,
    peft_config=lora_config
)

# 3) 학습된 모델 저장